# Cookbook Template

This gets you set up with a basic PluginInstanceRedisInterface that can communicate with your Nanome workspace from a Jupyter Notebook

Note that anything involving Redis is specific to the Cookbook project, and is not used for traditional plugins.

For more info on building regular plugins, see the 'Plugins' notebook.


## Setup.
1) Open up a Room in Nanome.
2) Navigate to the Stacks menu. If your Cookbook has been sucessfully deployed, you should have a "Cookbook" entry in your list
2) Start the Plugin, and the run button should say "Live". This mean the room is ready to accept requests from your notebook

In [1]:
import os
from nanome.beta.redis_interface import PluginInstanceRedisInterface

# Set up redis credentials
redis_host = os.environ.get("REDIS_HOST")
redis_port = os.environ.get("REDIS_PORT")
redis_password = os.environ.get("REDIS_PASSWORD")
redis_channel = ""  # Use room id code you receive when activating cookbook plugin

plugin_instance = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)
plugin_instance.connect()
print('Plugin Connected')

Plugin Connected


In [2]:
# Load a TYL into your workspace, and retrieve.
from nanome.api.structure import Complex

tyl_pdb = 'test_data/1tyl.pdb'
plugin_instance.send_files_to_load([tyl_pdb])


In [57]:
# Get new complex from workspace
ws = plugin_instance.request_workspace()
comp = ws.complexes[-1]

In [41]:
# Lets do some fun stuff
from nanome.api import structure
import numpy as np
from scipy.spatial import KDTree
import itertools

def get_neighboring_atoms(target_reference: structure.Complex, selected_atoms: list, site_size=6):
    """Use KDTree to find target atoms within site_size radius of selected atoms.
    
    returns a list of atoms within `site_size` angstroms of the ligand.
    """
    mol = next(
        mol for i, mol in enumerate(target_reference.molecules)
        if i == target_reference.current_frame)
    ligand_positions = [atom.position.unpack() for atom in selected_atoms]
    target_atoms = itertools.chain(*[ch.atoms for ch in mol.chains if not ch.name.startswith("H")])
    target_tree = KDTree([atom.position.unpack() for atom in target_atoms])
    target_point_indices = target_tree.query_ball_point(ligand_positions, site_size)
    near_point_set = set()
    for point_indices in target_point_indices:
        for point_index in point_indices:
            near_point_set.add(tuple(target_tree.data[point_index]))
    neighbor_atoms = []
    for targ_atom in mol.atoms:
        if targ_atom.position.unpack() in near_point_set:
            neighbor_atoms.append(targ_atom)
    return neighbor_atoms

In [69]:

from nanome.util import Color

# Make sure ligand is visible and deselected (We don't want to apply color scheme)
ligand_res = next(res for res in comp.residues if res.name.upper() == 'TYL')
for atom in ligand_res.atoms:
    atom.set_visible(True)
    atom.selected = False

# Use get neighboring atoms to detect pocket.
ligand_atoms = list(ligand_res.atoms)
ligand_pocket = get_neighboring_atoms(comp, ligand_atoms)

# Loop through protein, and show surface around pocket
non_ligand_residues = (comp_res for comp_res in comp.residues if comp_res.index != ligand_res.index)
for res in non_ligand_residues:
    # Show surface on ligand pocket
    res.ribboned = True
    for atom in res.atoms:
        if atom in ligand_pocket:
            atom.atom_mode = enums.AtomRenderingMode.Wire
            atom.set_visible(True)
            atom.surface_rendering = True
            atom.selected = True
        else:
            atom.atom_mode = enums.AtomRenderingMode.Wire
            atom.set_visible(True)
            atom.surface_rendering = False
            atom.selected = False

plugin_instance.update_structures_deep([comp])
plugin_instance.apply_color_scheme(enums.ColorScheme.BFactor, enums.ColorSchemeTarget.All)
